In [1]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git -b get_by_ee
!git clone https://github.com/shansen5/CA_Prisons.git
print( 'Done!' )

Cloning into 'ECHO_modules'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 154 (delta 80), reused 103 (delta 45), pack-reused 0
Receiving objects: 100% (154/154), 64.93 KiB | 5.41 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Cloning into 'CA_Prisons'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 17), reused 26 (delta 9), pack-reused 0
Unpacking objects: 100% (34/34), done.
Done!


In [2]:
%run ECHO_modules/DataSet.py
%run ECHO_modules/make_data_sets.py

### 1. Read the local spreadsheet with the carceral facilities

In [3]:
carc_data  = pd.read_csv( "CA_Prisons/reformatted_fixed_GG_manual_coding_hifld_prisons_to_FRS.csv", encoding='iso-8859-1',
                 dtype={"Total_FRS_IDS": "Int64"} )
registry_ids = carc_data['Total_FRS_IDS'].dropna().unique()
print( "Number of records in CSV = {}, number of non-NA ids = {}".format( len(carc_data), len(registry_ids)))


Number of records in CSV = 14289, number of non-NA ids = 4707


### 2. Look up the facilities in ECHO_EXPORTER.  The records found will be written to ECHO_EXPORTER-CA_Carceral.csv.

If you have run this notebook before and have the ECHO_EXPORTER-CA_Carceral.csv file
already generated, you can use the following cell to read it directly and
save some time compared to the full database query.  If you don't have the
CSV file, skip to the next cell to get the data from the database.

In [5]:
echo_data = pd.read_csv( "CA_Prisons/rECHO_EXPORTER-CA_Carceral.csv" )
echo_data.set_index( 'REGISTRY_ID', inplace=True)

FileNotFoundError: ignored

In [6]:
id_list = []
for id in registry_ids:
    id_list.append( id )
    
ds = DataSet( name="exporter", base_table="ECHO_EXPORTER",
            table_name="ECHO_EXPORTER", echo_type=None, idx_field="REGISTRY_ID" )
echo_data = ds.get_data_by_ee_ids( id_list )

echo_data.to_csv( "ECHO_EXPORTER-CA_Carceral.csv")

...
4707 ids were searched for
2402 program records were found


### 3. Look up the facilities in the program data sets.  The records found will be written to CSV files for each data set.

In [7]:
data_set_list = ['RCRA Violations', 'RCRA Inspections', 'RCRA Penalties',
                 'CAA Violations', 'CAA Inspections', 'CAA Penalties', 
                 'Greenhouse Gas Emissions', 'Toxic Releases',
                 'CWA Violations', 'CWA Inspections', 'CWA Penalties',
                 'SDWA Violations', 'SDWA Serious Violators', 'SDWA Site Visits',
                 'SDWA Enforcements', ] 
# You can run an individual data set, or just a few, by un-commenting and
# editing the following
# data_set_list = ['CWA Violations']
data_sets = make_data_sets( data_set_list )
last_flag = ''
pgm_ids = None
for pgm_name,ds in data_sets.items():
    print( pgm_name )
    if ( ds.echo_type == 'SDWA' ):
        echo_flag = 'SDWIS_FLAG'
    else:
        echo_flag = ds.echo_type + '_FLAG'
    # If the flag hasn't changed, the pgm_ids will still be the same.
    # Getting the pgm_ids from the echo_data's REGISTRY_ID takes a
    # long time, so we skip it if we can.
    if ( last_flag != echo_flag ):
        r_ids = echo_data[ echo_data[ echo_flag ] == 'Y' ].index
        pgm_ids = ds.get_pgm_ids( r_ids )
        last_flag = echo_flag
    program_data = ds.get_data_by_pgm_ids( pgm_ids )
    if ( program_data is not None):
        program_data.to_csv( pgm_name+'.csv' )

RCRA Violations


KeyboardInterrupt: ignored

Parse out REGISTRY_IDs in order to query ICIS federal enforcement and compliance table

In [46]:
ids = ""
for id in id_list[1:10]: # test on just 20 REGISTRY_IDs
    ids+="'"+str(id)+"',"
ids=ids[:-1] #remove trailing comma
ids=ids+",'110013272680'" #Texas Department of Corrections facility (for testing purposes)
ids

"'110011628756','110012969083','110064064382','110045674267','110043979086','110010111887','110050978527','110060265591','110045700924','110013272680'"

Get case numbers based on REGISTRY_IDs

In [51]:
facilities = None
try:
  #sql = 'select * from "CASE_FACILITIES" where "REGISTRY_ID" = \'110013272680\'' #Texas Department of Corrections facility (for testing purposes)
  sql = 'select * from "CASE_FACILITIES" where "REGISTRY_ID" in ('+ids+')'
  url= 'http://portal.gss.stonybrook.edu/echoepa/index.php?query=' # Old server: 'http://apps.tlt.stonybrook.edu/echoepa/index2.php?query='
  data_location=url+urllib.parse.quote_plus(sql) + '&pg'
  print(sql) # For debugging
  print(data_location)
  facilities = pd.read_csv(data_location)
except pd.errors.EmptyDataError:
  print("No facilities match!")
facilities

select * from "CASE_FACILITIES" where "REGISTRY_ID" in ('110011628756','110012969083','110064064382','110045674267','110043979086','110010111887','110050978527','110060265591','110045700924','110013272680')
http://portal.gss.stonybrook.edu/echoepa/index.php?query=select+%2A+from+%22CASE_FACILITIES%22+where+%22REGISTRY_ID%22+in+%28%27110011628756%27%2C%27110012969083%27%2C%27110064064382%27%2C%27110045674267%27%2C%27110043979086%27%2C%27110010111887%27%2C%27110050978527%27%2C%27110060265591%27%2C%27110045700924%27%2C%27110013272680%27%29&pg


,ACTIVITY_ID,CASE_NUMBER,REGISTRY_ID,FACILITY_NAME,LOCATION_ADDRESS,CITY,STATE_CODE,ZIP,PRIMARY_SIC_CODE,PRIMARY_NAICS_CODE
0,3600738616,06-2016-1212,110013272680,TDCJ W PACK UNIT,NaN,NaN,TX,NaN,NaN,NaN
1,2600045351,06-2011-1355,110013272680,TDCJ W PACK UNIT,NaN,NaN,TX,NaN,NaN,NaN


Use case numbers to get enforcement action details

In [59]:
#separate out case numbers in order to query the CASE_ENFORCEMENTS table
cases_list = list(facilities["CASE_NUMBER"].unique())
cases = ""
for case in cases_list:
  cases+="'"+case+"',"
cases=cases[:-1] #remove trailing comma
actions = None

#query the CASE_ENFORCEMENTS table
try:
  sql = 'select * from "CASE_ENFORCEMENTS" where "CASE_NUMBER" in ('+cases+')'
  url= 'http://portal.gss.stonybrook.edu/echoepa/index.php?query=' # Old server: 'http://apps.tlt.stonybrook.edu/echoepa/index2.php?query='
  data_location=url+urllib.parse.quote_plus(sql) + '&pg'
  print(sql) # For debugging
  print(data_location)
  actions = pd.read_csv(data_location)
except pd.errors.EmptyDataError:
  print("No actions found!")
actions

actions.to_csv('additional_enforcements.csv' )

select * from "CASE_ENFORCEMENTS" where "CASE_NUMBER" in ('06-2016-1212','06-2011-1355')
http://portal.gss.stonybrook.edu/echoepa/index.php?query=select+%2A+from+%22CASE_ENFORCEMENTS%22+where+%22CASE_NUMBER%22+in+%28%2706-2016-1212%27%2C%2706-2011-1355%27%29&pg


,ACTIVITY_ID,ACTIVITY_NAME,STATE_CODE,REGION_CODE,FISCAL_YEAR,CASE_NUMBER,CASE_NAME,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,ACTIVITY_STATUS_CODE,ACTIVITY_STATUS_DESC,ACTIVITY_STATUS_DATE,LEAD,CASE_STATUS_DATE,DOJ_DOCKET_NMBR,ENF_OUTCOME_CODE,ENF_OUTCOME_DESC,TOTAL_PENALTY_ASSESSED_AMT,TOTAL_COST_RECOVERY_AMT,TOTAL_COMP_ACTION_AMT,HQ_DIVISION,BRANCH,VOLUNTARY_SELF_DISCLOSURE_FLAG,MULTIMEDIA_FLAG,ENF_SUMMARY_TEXT
0,2600045351,TDCJW Pack Unit,NaN,6,2011,06-2011-1355,TDCJW Pack Unit,AFR,Administrative - Formal,CSU,Closed Superseded,05/26/2016,EPA,05/26/2016,NaN,ESU,Superseded by Another Enforcement Action,NaN,NaN,NaN,NaN,6EN-W,N,NaN,Administrative order issued for non compliance...
1,3600738616,Texas Department of Criminal Justice,NaN,6,2016,06-2016-1212,Texas Department of Criminal Justice,AFR,Administrative - Formal,CLS,Closed,10/25/2017,EPA,10/25/2017,NaN,ECN,Final Order No Penalty,NaN,NaN,NaN,NaN,6EN-W,N,NaN,"On May 26, 2016, EPA Region 6 issued an Admini..."


In [ ]:
""